# Data Manipulation with Pandas

## Context
NumPy is great for pure mathematical operations on matrices, but as an SRE, your data is messy. You receive CSV exports of billing data, JSON dumps of server inventory, and unstructured application logs. You need a tool that handles missing data, column names, and time-series alignments natively.

## Objectives
- Understand the core Pandas data structures: `Series` (1D) and `DataFrame` (2D).
- Learn how to clean messy infrastructure data (handling NaN values and dropping duplicates).
- Perform SQL-like aggregations (groupby) and time-series resampling on server metrics.

## Expected Outcome
- The ability to take a raw, messy csv of server logs, clean it, group it by microservice, and output actionable aggregated metrics.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### 1. Pandas Data Structures
Think of a `DataFrame` exactly like a SQL table or an Excel spreadsheet. It has rows, named columns, and an index. A single column extracted from a DataFrame is called a `Series`.

In [ ]:
# Creating a DataFrame from a dictionary (Simulating an active server inventory)
inventory_data = {
    'Hostname': ['web-01', 'web-02', 'db-01', 'worker-01'],
    'Role': ['Frontend', 'Frontend', 'Database', 'Background'],
    'CPU_Cores': [4, 4, 16, 8],
    'Status': ['Active', 'Active', 'Maintenance', 'Active']
}
df_inventory = pd.DataFrame(inventory_data)
print("Server Inventory DataFrame:\n")
print(df_inventory)

# Extracting a single Series (just the Hostnames)
hostnames = df_inventory['Hostname']
print("\nHostname Series (type: ", type(hostnames), "):")
print(hostnames)

### 2. Data Cleaning (Handling Messy Logs)
In the real world, instrumentation fails. Metrics get dropped. You will frequently encounter `NaN` (Not a Number) values. Pandas gives you tools to handle these safely.

In [ ]:
# Simulating incomplete metrics reporting from an agent cluster
messy_metrics = pd.DataFrame({
    'Server': ['app-1', 'app-2', 'app-3', 'app-1'],  # Notice the duplicate app-1
    'Memory_Usage_%': [85.5, np.nan, 42.1, 85.5],    # Missing value for app-2
    'Disk_IO_MBps': [120, 15, np.nan, 120]
})
print("Raw Messy Data:\n", messy_metrics)

# 1. Drop exact duplicate rows (retries from the metrics agent)
df_clean = messy_metrics.drop_duplicates()
print("\nAfter dropping duplicates:\n", df_clean)

# 2. Handle missing data
# Option A: Forward-fill or interpolate (dangerous for critical alerting, good for smooth charts)
# Option B: Fill with a safe default (e.g., 0)
# Option C: Drop the row entirely
df_filled = df_clean.fillna(
    {'Memory_Usage_%': df_clean['Memory_Usage_%'].mean(),  # Impute with average
     'Disk_IO_MBps': 0}                                    # Assume 0 if not reported
)
print("\nAfter handling missing values:\n", df_filled)

### 3. Data Transformation
Often you need to derive new metrics from existing ones, or rename columns to match a standard schema (e.g., standardizing on Prometheus labels).

In [ ]:
# Renaming columns to standard labels
df_renamed = df_inventory.rename(columns={'Hostname': 'instance', 'Role': 'service'})

# Creating a new derived column (e.g., mapping Cores to an estimated monthly cost)
core_cost_per_month = 15.00
df_renamed['Estimated_Cost_$'] = df_renamed['CPU_Cores'] * core_cost_per_month

print(df_renamed)

### 4. Grouping and Aggregation
`groupby` operations are the backbone of exploratory data analysis. They function exactly like SQL `GROUP BY` statements.

In [ ]:
# Simulating 10,000 requests processed by 3 microservices
np.random.seed(42)
requests_df = pd.DataFrame({
    'Service': np.random.choice(['Auth', 'Payments', 'Search'], 10000),
    'Latency_ms': np.random.gamma(2, 50, 10000),
    'Status_Code': np.random.choice([200, 400, 500], 10000, p=[0.9, 0.08, 0.02])
})

# Which service is the slowest on average?
avg_latency = requests_df.groupby('Service')['Latency_ms'].mean().sort_values(ascending=False)
print("Average Latency by Service:\n", avg_latency)

# Complex aggregation: Get count, mean latency, and 95th percentile by service
def p95(x):
    return x.quantile(0.95)

service_metrics = requests_df.groupby('Service').agg({
    'Latency_ms': ['count', 'mean', p95],
    'Status_Code': lambda x: (x == 500).sum()  # Count 500 errors
})
print("\nDetailed Service Metrics:\n", service_metrics)

### 5. Time-Series Data Handling
Pandas was originally built for financial modeling, so its time-series capabilities are unmatched. This is vital for analyzing Prometheus or Datadog exports.

In [ ]:
# Generate a time series of CPU Temperature readings taken every 10 seconds for 1 hour
time_index = pd.date_range(start='2025-02-25 00:00:00', periods=360, freq='10S')
cpu_temps = np.random.normal(loc=45, scale=5, size=360) + np.linspace(0, 15, 360) # Slowly overheating

temp_df = pd.DataFrame({'Temp_C': cpu_temps}, index=time_index)

print("Raw 10-second resolution data (first 3 rows):\n", temp_df.head(3))

# Resampling: Downsample from 10-second data to 5-minute averages
# This is exactly what Prometheus does when you zoom out on a Grafana dashboard!
downsampled = temp_df.resample('5T').mean()
print("\nDownsampled 5-minute averages:\n", downsampled)

# Simple built-in plotting
plt.figure(figsize=(10, 4))
temp_df['Temp_C'].plot(alpha=0.5, label='Raw (10s)')
downsampled['Temp_C'].plot(color='red', linewidth=2, label='Avg (5min)')
plt.title("Server CPU Temperature Analysis")
plt.ylabel("Degrees (C)")
plt.legend()
plt.grid(True)
plt.show()

### 6. Merging and Joining (Correlating Data)
SREs rarely get all answers from one system. You might need to join billing export data (CSV) with current usage metrics (API JSON) to find the cost per request.

In [ ]:
# The Inventory Data (from earlier)
print("Inventory:\n", df_renamed[['instance', 'Estimated_Cost_$']])

# New Metrics Data (e.g., pulled from Thanos)
usage_data = pd.DataFrame({
    'instance': ['web-01', 'web-02', 'db-01'],
    'Total_Requests': [1500000, 1450000, 8000000]
})

# Merge them (Inner Join by default)
merged_df = pd.merge(df_renamed, usage_data, on='instance', how='left')

# Calculate cost per million requests
merged_df['Cost_Per_1M_Req'] = (merged_df['Estimated_Cost_$'] / merged_df['Total_Requests']) * 1000000

print("\nMerged Business Correlation:\n", merged_df[['instance', 'Role', 'Cost_Per_1M_Req']])